# The school mathematics behind SVM

The support-vetor machine (SVM) algorithm is essentially a binary classifier of two clusters of points. It is a non-statistical method, and the mathematics behind the verval description of how it picks the "middle" hyperplane seperating the two groups isn't actually that difficult. It's just Eucledean geometry, or at worst, the first-month amount of vector calculus. Not many blogs on machine learning describes this simple math behind it, so I'll give it a go for my future students.

## The first month of vector calculus

We fist need a description of a hyperplane. Its "slope" is determined by a normal vector $\textbf{n}$,
$$\textbf{n} = <n_1, n_2, n_3>.$$
(I'll not normalize it because many SVM documents seem to prefer not normalizing it, and I don't wish to create confusion in the community.)

However, this is not enough to determine its position in the space. To determine the position, we need a constant $b$ derived from
$$n_1 (x - x_0) + n_2 (y - y_0) + n_3 (z - z_0) = 0$$
where $P = (x, y, z)$ is any point on the plane and $P_0 = (x_0, y_0, z_0)$ is another point on the plane so that
$$n \cdot \overset{\rightarrow}{OP} = b.$$

Given $\textbf{n}$ and $b$, there is a canonical way to choose $P_0$, simply by drawing a line from the origin $O$ in the direction of $\textbf{n}$. $P_0$ is where the line hits the plane (or the origin itself if the plane contains the origin!), and it can be calculated using $b$.
- We only need to find the length of $\overset{\rightarrow}{OP_0}$.
- This is simply
$$\overset{\rightarrow}{OP_0} \cdot \frac{\textbf{n}}{||\textbf{n}||}$$
in formula.
- But $P_0$ is again a point on the plane, thus
$$\overset{\rightarrow}{OP_0} \cdot \textbf{n} = b.$$
- Therefore $$||\overset{\rightarrow}{OP_0}|| = \frac{b}{||\textbf{n}||}.$$
- So the canonical choice of $P_0$ is described as
$$\overset{\rightarrow}{OP_0} = \bigg( \frac{b}{||\textbf{n}||} \bigg) \frac{\textbf{n}}{||\textbf{n}||}.$$

Now we are ready to determine whether a given sample point $Q$ outside the plane is on one side of the plane or on the other side. (For math geeks, yes, the plane is an orientable surface, and we've chosen an orientation as soon as $\textbf{n}$ was given.)

If $Q$ is on the side that the normal vector $\textbf{n}$ sticks to, we'll say $Q$ is on the positive side (of the plane). Otherwise we'll say it's on the negative side.

And we know that the dot product tells us whether two non-zero vetors generally align the same direction (having an acute angle with the positive dot product) or, if not perpendicular, the opposite direction (having a reflex angle with the negative dot product).

Our two vectors in this scenario will be $\overset{\rightarrow}{P_0Q}$ and $\textbf{n}$. Note that
$$\overset{\rightarrow}{P_0Q} = \overset{\rightarrow}{OQ} - \overset{\rightarrow}{OP_0}$$
by the, well, the first week of vector calculus.

Combining with $$\overset{\rightarrow}{OP_0} = \bigg( \frac{b}{||\textbf{n}||} \bigg) \frac{\textbf{n}}{||\textbf{n}||},$$
we have
\begin{align*}
\overset{\rightarrow}{P_0Q} \cdot \textbf{n} &= \bigg( \overset{\rightarrow}{OQ} - \overset{\rightarrow}{OP_0} \bigg) \cdot \textbf{n} \\
&= \Bigg( \overset{\rightarrow}{OQ} - \bigg( \frac{b}{||\textbf{n}||} \bigg) \frac{\textbf{n}}{||\textbf{n}||} \Bigg) \cdot \textbf{n} \\
&= \overset{\rightarrow}{OQ} \cdot \textbf{n} - b.
\end{align*}

So we have
$$\text{$Q$ is on the positive side if } \overset{\rightarrow}{OQ} \cdot \textbf{n} - b > 0,$$
$$\text{$Q$ is on the negative side if } \overset{\rightarrow}{OQ} \cdot \textbf{n} - b < 0.$$

Note also that the (signed) distance between the external point $Q$ and the plane is simply the version with the unit normal vector $\frac{\textbf{n}}{||\textbf{n}||}$,
$$\overset{\rightarrow}{P_0Q} \cdot \frac{\textbf{n}}{||\textbf{n}||} = \overset{\rightarrow}{OQ} \cdot \frac{\textbf{n}}{||\textbf{n}||} - \frac{b}{||\textbf{n}||}.$$

## The actual algorithm in SVM

Here's the actual implementation of the algorithm. The goal is to find nice $\textbf{n}$ and $b$ that does the binary classification job done.

### Hard-margin

Suppose the training data is linearly separable, i.e., there is definitely a hyperplane that splits the two clusters exactly.

During the process, we normalize $\textbf{n}$ and $b$ in a way that not many pure mathematicians would usually prefer (like taking the unit $\textbf{n}$ or setting $b = 1$). Instead, we pose a condition so that
- Suppose $Q_1$ is the closest point on the positive side and $Q_2$ is the closest point on the negative side.
- Since our plane should be placed exactly right in the middle of the two, the distance from the plane to $Q_1$ should be the same as the distance to $Q_2$.
- Take that to be $\frac{1}{||\textbf{n}||}$, i.e.,
$$\overset{\rightarrow}{OQ_1} \cdot \textbf{n} - b  = 1,$$
$$\overset{\rightarrow}{OQ_2} \cdot \textbf{n} - b  = -1$$
- This is where many literatures on SVM say the distance between two clusters is $\frac{2}{||\textbf{n}||}$.

We wish to find $\textbf{n}$ and $b$ such that this $\frac{2}{||\textbf{n}||}$-gap is as wide as possible, i.e., $||\textbf{n}||$ is as small as possible.

Note that, by this normalization constraint, for all other sample points $Q_i$, we have
$$\overset{\rightarrow}{OQ} \cdot \textbf{n} - b \geq 1 \text{ if $Q_i$ is on the positive side},$$
$$\overset{\rightarrow}{OQ} \cdot \textbf{n} - b \leq -1 \text{ if $Q_i$ is on the negative side}.$$

Since each sample $Q_i$ should be given in the form $(x_i, y_i)$ where $x_i$ is the vector in the feature space and $y_i$ is the label ($1$ or $-1$), in many literatures, we often find the same thing phrased as
$$\textbf{n} \cdot x_i - b \geq 1 \text{ if $y_i = 1$},$$
$$\textbf{n} \cdot x_i  - b \leq -1 \text{ if $y_i = -1$}.$$

The advantage of having the value of the label $1$ or $-1$ is that the above inequalities can be coded in a single line,
$$y_i (\textbf{n} \cdot x_i - b) \geq 1.$$

Then the optimization problem asks for the following:
$$\text{Minimize } ||\textbf{n}|| \text{ such that } y_i (\textbf{n} \cdot x_i - b) \geq 1 \text{ for all } i = 1, ..., n,$$
or equivalently,
$$\text{Minimize } \frac{1}{2}||\textbf{n}||^2 \text{ such that } y_i (\textbf{n} \cdot x_i - b) \geq 1 \text{ for all } i = 1, ..., n,$$

The solution $\textbf{n}$ and $b$ to this problem give our classifier
$$x \mapsto y_{\text{predict}} = \text{sgn}(\textbf{n} \cdot x - b).$$

The normal vectors from the plane to the closest points are called *support vectors*.

### Soft-margin

Not every clusters can be distinguished by a hyperplane. Here's how we loosen the impossible hard-margin condition in this case.

So what's it like to be the case where the linear classification fails?
- There is $Q_i = (x_i, y_i)$ such that
$$y_i (\textbf{n} \cdot x_i - b) \leq 1,$$
i.e.,
$$1 - y_i (\textbf{n} \cdot x_i - b) \geq 0.$$

Based on this observation, we introduce the *hinge loss* funciton
$$\text{max}\big(0, 1 - y_i (\textbf{n} \cdot x_i - b) \big)$$
which we wish to see as many 0's as much as $i$ spans.

Then we wish to minimize
$$\Bigg( \frac{1}{n} \sum_{i=1}^n \text{max}\big(0, 1 - y_i (\textbf{n} \cdot x_i - b) \big) \Bigg) + \lambda ||\textbf{n}||^2.$$